# Find Best Flights - A Star

#### load libraries 

In [36]:
import heapq
from pandas import read_csv

#### Set DataFrame

In [37]:
df = read_csv('Flight_Data.csv')

print(df)

### A star Configs

با استفاده از ضرایب خواسته شده وزن هر مسیر مشخص می شود و بر اساس آن مسیریابی انجام می شود.

In [1]:
Time = 0
Distance = 1
Price = 0

### Heuristic weight

این ضریب تاثیر وزن تابع Heuristic را مشخص می کند.

In [2]:
heuristic_weight = 2

### Weight calculator for A star

In [25]:
def calculate(distance, flyTime, price, Source, Destination):
    dijkstra = round(Time * flyTime + Distance * distance + Price * price)
    heuristic = round(
        (((Source.Latitude - Destination.Latitude) ** 2) +
         ((Source.Longitude - Destination.Longitude) ** 2) +
         ((Source.Altitude - Destination.Altitude) ** 2)) ** 0.5)
    return round(dijkstra + heuristic_weight * heuristic)

#### Node CLass

In [26]:
class Node:
    def __init__(self, Airport, Airport_Country, Airport_City, Latitude, Longitude, Altitude):
        # Source data
        self.Airport = Airport
        self.Country = Airport_Country
        self.City = Airport_City
        self.Latitude = Latitude
        self.Longitude = Longitude
        self.Altitude = Altitude

    def __str__(self):
        return self.City + "-" + self.Airport + ", " + self.Country

#### Edge Class

In [27]:
class Edge:
    def __init__(self, Source, Destination, FinalDestination, distance, flyTime, price):
        self.price = round(price, 2)
        self.flyTime = round(flyTime, 2)
        self.distance = round(distance, 2)
        self.destination = Destination
        self.source = Source
        self.weight = calculate(distance, flyTime, price, Source, FinalDestination)

    def __str__(self):
        return "From: " + str(self.source) + "\nTo: " + str(self.destination) + "\nDuration: " + str(
            self.distance) + "km\nTime: " + str(self.flyTime) + "h\nPrice: " + str(self.price) + "$"

    def __gt__(self, other):
        return self.weight > other.weight

#### Find Unique Nodes

In [28]:
nodes = set()
for data in df.values:
    nodes.add(data[3] + "," + data[1] + "," + data[4] + "," + str(data[5]) + "," + str(data[6]) + "," + str(data[7]))
    nodes.add(data[8] + "," + data[2] + "," + data[9] + "," + str(data[10]) + "," + str(data[11]) + "," + str(data[12]))
    
print(nodes)

### Make Node classes from nodes (set)
used for create Edge class

In [29]:
NodeClass = []
for i in nodes:
    data = i.split(',')
    NodeClass.append(Node(data[1], data[2], data[0], float(data[3]), float(data[4]), float(data[5])))
    
print(NodeClass)

#### Getting input
It's like "Imam Khomeini International Airport - Raleigh Durham International Airport"

In [30]:
trip = input().split(' - ')

print(trip)

Imam Khomeini International Airport - Raleigh Durham International Airport


### Make Node classes from nodes (set)
used for create Edge class

In [31]:
nodeSearch = [x.split(',')[1] for x in nodes]

SourcePath = nodeSearch.index(trip[0])
DestinationPath = nodeSearch.index(trip[1])

print(SourcePath)
print(DestinationPath)

### Made edge lists
edge list used for dijkstra proccess 

In [33]:
DestinationPathClass = NodeClass[DestinationPath]

nodes = list(nodes)
edges = []
EdgeClass = []

for i in df.values:
    source = nodes.index(i[3] + "," + i[1] + "," + i[4] + "," + str(i[5]) + "," + str(i[6]) + "," + str(i[7]))
    destination = nodes.index(i[8] + "," + i[2] + "," + i[9] + "," + str(i[10]) + "," + str(i[11]) + "," + str(i[12]))
    edges.append([source, destination,
                  Edge(NodeClass[source], NodeClass[destination], DestinationPathClass, i[13],i[14], i[15])
                 ])

## Dijkstra find Shortest Path function

نحوه کار این تابع به این شکل است که هر بار وارد گره ای می شود تمامی مسیر های خارج شده از آن را به minHeap اضافه می کند و پس از آن کوتاه ترین مسیر را از minHeap انتخواب و به گره بعدی می رود. بدین ترتیب هر بار به گره ای برسد چون هر بار کمترین مسیر را انتخواب کرده، کمتر مسیر است پس دفعات بعدی که به آن گره برسد را در نظر نمی گیرد. در مواقعی که به یک گره جدید می رسد مسیر را ذخیره و درصورتی که به مقصد نهایی برسد اجرای تابع قطع می شود و مسیر را برمیگرداند. 

In [34]:
def ShortestPath(N, Edges, Src, Dis):
    adj = {}
    for j in range(N):
        adj[j] = []

    for s, d, edge in Edges:
        adj[s].append([d, edge.weight, edge])

    shortest = {}  # map vertex -> dict of the shortest path

    minHeap = [[0, Src, [Src], []]]

    while minHeap:
        w1, n1, way1, edge1 = heapq.heappop(minHeap)
        if n1 in shortest:
            continue
        shortest[n1] = [way1, edge1]
        if n1 == Dis:
            break

        for n2, w2, edge2 in adj[n1]:
            if n2 not in shortest:
                heapq.heappush(minHeap, [w1 + w2, n2, way1 + [n2], edge1 + [edge2]])

    for k in range(N):
        if k not in shortest:
            shortest[k] = -1

    return shortest

#### Preparing Output

In [35]:
Duration = 0
FlyTime = 0
Price = 0
index = 1
for i in ShortestPath(len(nodes), edges, SourcePath, DestinationPath)[nodeSearch.index(trip[1])][1]:
    print('Flight #' + str(index) + ":")
    index += 1
    Duration += round(i.distance)
    Time += round(i.flyTime)
    Price += round(i.price)
    print(str(i), "\n----------------------------")

print("total Price: " + str(Price) + "$")
print("total Duration: " + str(Duration) + "KM")
print("total Fly Time : " + str(Time) + "H")


Flight #1:
From: Tehran-Imam Khomeini International Airport, Iran
To: Istanbul-Atatürk International Airport, Turkey
Duration: 2040.98km
Time: 3.04h
Price: 990.49$ 
----------------------------
Flight #2:
From: Istanbul-Atatürk International Airport, Turkey
To: Washington-Washington Dulles International Airport, United States
Duration: 8413.02km
Time: 10.95h
Price: 4176.51$ 
----------------------------
Flight #3:
From: Washington-Washington Dulles International Airport, United States
To: Raleigh-durham-Raleigh Durham International Airport, United States
Duration: 360.72km
Time: 0.95h
Price: 150.36$ 
----------------------------
total Price: 5317$
total Duration: 10815KM
total Fly Time : 15H


# End
## other Resource
https://youtu.be/XEb7_z5dG3c?si=hE5rwWGypR5ytY7_